In [0]:
sc

<SparkContext master=spark://10.139.64.4:7077 appName=Databricks Shell>

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df = spark.read.format("csv").option("header","true").option("inferschema","true").load("dbfs:/Volumes/azuredatabme/default/sales/sales_data_sample.csv")

In [0]:
df.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

In [0]:
df.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

Customer Sales Contribution

In [0]:
df.createOrReplaceTempView("productssales")

In [0]:
spark.sql("""
          select 
            CUSTOMERNAME,
            SUM(SALES) AS Sum_of_sales,
            SUM(SALES)*100/(select SUM(SALES) from productssales) AS Customer_contribution_pct 
            from productssales 
            group by CUSTOMERNAME
            """).show(10)

+--------------------+------------------+-------------------------+
|        CUSTOMERNAME|      Sum_of_sales|Customer_contribution_pct|
+--------------------+------------------+-------------------------+
| Suominen Souveniers|113961.14999999997|       1.1359051720526854|
|  Amica Models & Co.| 94117.26000000002|        0.938111649570291|
|Collectables For ...|          81577.98|       0.8131266612140238|
|         CAF Imports|          49642.05|      0.49480600490867305|
|   giftsbymail.co.uk| 78240.83999999998|       0.7798637941241083|
|       Rovelli Gifts|137955.72000000003|       1.3750705030815515|
|     Lyon Souveniers| 78570.34000000001|       0.7831480778838931|
|   La Rochelle Gifts|          180124.9|       1.7953908461389938|
| L'ordine Souveniers|142601.33000000002|       1.4213755151522411|
|Signal Collectibl...| 50218.51000000001|       0.5005518568545467|
+--------------------+------------------+-------------------------+
only showing top 10 rows



## Find seasonal sales trends

In [0]:

%sql SELECT TOTAL_SALES, MONTH, QTR, 
CASE
 WHEN QTR=1 THEN 'Winter'
 WHEN QTR =2 THEN 'Spring'
  WHEN QTR =3 THEN 'Summer' 
   WHEN QTR =4 THEN 'Fall'
   END AS SEASON FROM(select round(sum(sales),2) AS TOTAL_SALES,qtr_id as QTR, month_id as month from productssales group by month,qtr_id order by qtr_id,month)

TOTAL_SALES,MONTH,QTR,SEASON
785874.44,1,1,Winter
810441.9,2,1,Winter
754501.39,3,1,Winter
669390.96,4,2,Spring
923972.56,5,2,Spring
454756.78,6,2,Spring
514875.97,7,3,Summer
659310.57,8,3,Summer
584724.27,9,3,Summer
1121215.22,10,4,Fall


Databricks visualization. Run in Databricks to view.

In [0]:

spark.sql('select round(sum(sales)),qtr_id as QTR, month_id as month from productssales group by month,qtr_id order by qtr_id,month').show()

+--------------------+---+-----+
|round(sum(sales), 0)|QTR|month|
+--------------------+---+-----+
|            785874.0|  1|    1|
|            810442.0|  1|    2|
|            754501.0|  1|    3|
|            669391.0|  2|    4|
|            923973.0|  2|    5|
|            454757.0|  2|    6|
|            514876.0|  3|    7|
|            659311.0|  3|    8|
|            584724.0|  3|    9|
|           1121215.0|  4|   10|
|           2118886.0|  4|   11|
|            634679.0|  4|   12|
+--------------------+---+-----+



In [0]:
%sql select sum(sales) as Total_Sales from productssales

Total_Sales
1.003262885000001E7


Databricks visualization. Run in Databricks to view.

In [0]:

%sql select avg(monthlysales) as average_monthly_sales from(select sum(sales) as monthlysales ,month_id as Total_Sales from productssales group by month_id)

average_monthly_sales
836052.4041666667


Databricks visualization. Run in Databricks to view.

In [0]:
%sql 
SELECT round(SUM(PROFIT),2) as Total_profit FROM (select PRICEEACH,QUANTITYORDERED,SALES, (SALES-PRICEEACH*QUANTITYORDERED) as PROFIT from productssales)

Total_profit
1741742.06


Databricks visualization. Run in Databricks to view.

In [0]:
%sql select PRODUCTLINE, SUM(QUANTITYORDERED) AS TOTAL_QUANTITY FROM productssales group by PRODUCTLINE ORDER BY PRODUCTLINE

PRODUCTLINE,TOTAL_QUANTITY
Classic Cars,33992
Motorcycles,11663
Planes,10727
Ships,8127
Trains,2712
Trucks and Buses,10777
Vintage Cars,21069


Databricks visualization. Run in Databricks to view.

In [0]:
%sql select CASE 
WHEN MONTH_ID=1 THEN 'January'
WHEN MONTH_ID=2 THEN 'February'
WHEN MONTH_ID=3 THEN 'March'
WHEN MONTH_ID=4 THEN 'April'
WHEN MONTH_ID=5 THEN 'May'
WHEN MONTH_ID=6 THEN 'June'
WHEN MONTH_ID=7 THEN 'July'
WHEN MONTH_ID=8 THEN 'August'
WHEN MONTH_ID=9 THEN 'September'
WHEN MONTH_ID=11 THEN 'November'
WHEN MONTH_ID=10 THEN 'October'
WHEN MONTH_ID=12 THEN 'December'
end as Month, SUM(QUANTITYORDERED) AS QTY_PER_MONTH FROM productssales group by MONTH_ID ORDER BY MONTH_ID

Month,QTY_PER_MONTH
January,7997
February,7903
March,7585
April,6704
May,8992
June,4620
July,4899
August,6538
September,5681
October,10998


Databricks visualization. Run in Databricks to view.

In [0]:
%sql select CUSTOMERNAME as CustomerName,SUM(SALES) AS Sum_of_sales,SUM(SALES)*100/(select SUM(SALES) from productssales) AS Customer_contribution_pct from productssales group by CUSTOMERNAME order by Sum_of_sales desc limit 10

CustomerName,Sum_of_sales,Customer_contribution_pct
Euro Shopping Channel,912294.1100000002,9.093270803095633
Mini Gifts Distributors Ltd.,654858.06,6.527282826773756
"Australian Collectors, Co.",200995.40999999997,2.0034171801341953
Muscle Machine Inc,197736.93999999997,1.9709384544809485
La Rochelle Gifts,180124.9,1.7953908461389938
"Dragon Souveniers, Ltd.",172989.68000000008,1.7242707029872821
Land of Toys Inc.,164069.44000000003,1.6353584135627608
The Sharp Gifts Warehouse,160010.26999999996,1.594898728861078
"AV Stores, Co.",157807.80999999997,1.5729457588775428
"Anna's Decorations, Ltd",153996.13000000003,1.5349529251249026


Databricks visualization. Run in Databricks to view.

In [0]:
df.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

# Calculate top 3 products based on total number of sales in each quarter

In [0]:
%sql select sum_sales,productcode,qtr_id,ranking from(select productcode,sum_sales,qtr_id,rank() over(partition by qtr_id order by sum_sales desc) as ranking from(select sum(sales) as sum_sales,productcode,qtr_id from productssales group by  qtr_id,productcode order by qtr_id, sum_sales desc))where ranking<4

sum_sales,productcode,qtr_id,ranking
54246.03999999999,S18_3232,1,1
42518.53999999999,S12_1108,1,2
40338.53999999999,S10_1949,1,3
70189.43000000001,S18_3232,2,1
52808.69,S10_4698,2,2
38806.13,S18_2238,2,3
50352.01,S18_3232,3,1
39477.92,S12_1099,3,2
38424.0,S10_1949,3,3
113457.93999999999,S18_3232,4,1


# Categorize products as Hot or Cold based on their ORDER Quantity in each quarter,using a CASE WHEN statement

In [0]:
# %sql 
# select qtr_id,total_quantity,productline,case 
# when ranking<4 then 'HOT'
# when ranking<6 then 'MEDIUM'
# when ranking<8 then 'COLD'
# END AS CATEGORY FROM
# (select ProductLine,total_quantity,qtr_id,rank() over (partition by qtr_id order by total_quantity desc)as ranking from(select ProductLine,QTR_ID,sum(QUANTITYORDERED) as total_quantity from productssales group by ProductLine, QTR_ID ORDER BY QTR_ID))

qtr_id,total_quantity,productline,CATEGORY
1,7784,Classic Cars,HOT
1,5481,Vintage Cars,HOT
1,2671,Motorcycles,HOT
1,2625,Planes,MEDIUM
1,2163,Ships,MEDIUM
1,2068,Trucks and Buses,COLD
1,693,Trains,COLD
2,6447,Classic Cars,HOT
2,4030,Vintage Cars,HOT
2,2773,Planes,HOT


In [0]:
%sql
SELECT 
    productLine,
    quantity,
    qtr_id,
    CASE
        WHEN quantity = max_quantity THEN 'hot'
        WHEN quantity = min_quantity THEN 'cold'
        ELSE 'medium'
    END AS category
FROM (
    SELECT 
        productLine,
        SUM(QUANTITYORDERED) AS quantity,
        qtr_id,
        MAX(SUM(QUANTITYORDERED)) OVER (PARTITION BY qtr_id) AS max_quantity,
        MIN(SUM(QUANTITYORDERED)) OVER (PARTITION BY qtr_id) AS min_quantity
    FROM productssales
    GROUP BY productLine, qtr_id
) AS fac GROUP BY productLine,
    quantity,
    qtr_id,category having category='hot' or category='cold'


productLine,quantity,qtr_id,category
Classic Cars,7784,1,hot
Trains,693,1,cold
Classic Cars,6447,2,hot
Trains,439,2,cold
Classic Cars,6190,3,hot
Trains,531,3,cold
Trains,1049,4,cold
Classic Cars,13571,4,hot


#### Identify product with outliers sales in each quarter 

In [0]:
%sql
WITH Quartiles as (
  select QTR_ID,PRODUCTCODE,
  PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY SALES) as Q1,
  PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY SALES) as Q3
from productssales
group by QTR_ID,PRODUCTCODE
),

OutlierBounds as (
  select QTR_ID,PRODUCTCODE,Q1,Q3,
  (Q3 + 1.5 * (Q3 - Q1)) as UpperBound,
  (Q1 + 1.5 * (Q3 - Q1)) as LowerBound
from Quartiles
)

select s.QTR_ID,s.PRODUCTCODE,s.SALES,ob.UpperBound,ob.LowerBound
from productssales s
join OutlierBounds ob
on s.QTR_ID = ob.QTR_ID and s.PRODUCTCODE = ob.PRODUCTCODE
where s.SALES > ob.UpperBound or s.SALES < ob.LowerBound

QTR_ID,PRODUCTCODE,SALES,UpperBound,LowerBound
1,S18_2248,1331.1,5329.8375,3739.6125
3,S24_4278,1666.35,2469.05625,2130.43875
1,S12_3148,4071.85,8487.506249999999,6785.41875
1,S18_2625,6981.0,3140.1462500000007,2405.4637500000003
3,S18_2625,1550.72,2398.3500000000004,2035.0500000000002
1,S24_2841,1611.4,4867.526249999999,3836.1437499999997
2,S24_3420,1254.6,2878.93,2336.8599999999997
1,S32_2206,1826.96,2930.1837499999992,2439.94125
4,S50_1341,2725.47,3805.5299999999997,2930.2699999999995
1,S10_1678,3207.12,5607.65,4512.99


#### Moving 3-months sales average for each product

In [0]:
%sql
select 
	year_id,
  month_id,
  productcode,
  sales,
	avg(sales) over(partition by productcode order by year_id, month_id rows between 3 preceding and current row) as Moving_avg_3_months
	from productssales
  order by productcode, year_id, month_id

year_id,month_id,productcode,sales,Moving_avg_3_months
2003,2,S10_1678,2871.0,2871.0
2003,5,S10_1678,2765.9,2818.45
2003,7,S10_1678,3884.34,3173.7466666666664
2003,8,S10_1678,3746.7,3316.9849999999997
2003,10,S10_1678,5205.27,3900.5525
2003,10,S10_1678,3479.76,4079.0175000000004
2003,11,S10_1678,2497.77,3732.3750000000005
2003,11,S10_1678,5512.32,4173.780000000001
2003,12,S10_1678,2168.54,3414.5975
2004,1,S10_1678,4708.44,3721.7675


#### Rolling 3_month Sales Average for each product (both are same)

In [0]:
%sql
select 
	year_id,
  month_id,
  productcode,
  sales,
	avg(sales) over(partition by productcode order by year_id, month_id rows between 3 preceding and current row) as Moving_avg_3_months
	from productssales
  order by productcode, year_id, month_id

year_id,month_id,productcode,sales,Moving_avg_3_months
2003,2,S10_1678,2871.0,2871.0
2003,5,S10_1678,2765.9,2818.45
2003,7,S10_1678,3884.34,3173.7466666666664
2003,8,S10_1678,3746.7,3316.9849999999997
2003,10,S10_1678,5205.27,3900.5525
2003,10,S10_1678,3479.76,4079.0175000000004
2003,11,S10_1678,2497.77,3732.3750000000005
2003,11,S10_1678,5512.32,4173.780000000001
2003,12,S10_1678,2168.54,3414.5975
2004,1,S10_1678,4708.44,3721.7675


#### 11. Yearly Sales performance for each territory 

In [0]:
%sql
select 
  year_id, 
  territory,
  sum(sales) as total_sales
from productssales
group by year_id, territory
order by territory, year_id

year_id,territory,total_sales
2003,APAC,253134.45000000004
2004,APAC,345307.8299999998
2005,APAC,147679.55000000002
2003,EMEA,1660314.5300000005
2004,EMEA,2389128.889999999
2005,EMEA,929828.9899999998
2003,Japan,243773.18
2004,Japan,168479.09999999992
2005,Japan,42920.939999999995
2003,NA,1359757.3799999997


Databricks data profile. Run in Databricks to view.

In [0]:
%sql
select 
  year_id, 
  territory,
  sum(sales) as total_sales,
  avg(sum(sales)) over (partition by territory order by year_id rows between unbounded preceding and current row) as avg_sale,
  max(sum(sales)) over (partition by territory order by year_id rows between unbounded preceding and current row) as max_sale
from productssales
group by year_id, territory
order by territory, year_id

year_id,territory,total_sales,avg_sale,max_sale
2003,APAC,253134.45000000004,253134.45000000004,253134.45000000004
2004,APAC,345307.8299999998,299221.1399999999,345307.8299999998
2005,APAC,147679.55000000002,248707.2766666666,345307.8299999998
2003,EMEA,1660314.5300000005,1660314.5300000005,1660314.5300000005
2004,EMEA,2389128.889999999,2024721.71,2389128.889999999
2005,EMEA,929828.9899999998,1659757.47,2389128.889999999
2003,Japan,243773.18,243773.18,243773.18
2004,Japan,168479.09999999992,206126.13999999996,243773.18
2005,Japan,42920.939999999995,151724.40666666665,243773.18
2003,NA,1359757.3799999997,1359757.3799999997,1359757.3799999997


### Monthly Totalordered performances of each ProductLine - sum , Median, Variance, AVerage, Max

In [0]:
%sql
select 
  month_id, 
  productline,
  sum(quantityordered) as total_quantity_ordered,
  avg(sum(quantityordered)) over (partition by productline order by month_id rows between unbounded preceding and current row) as avg_sale,
  percentile_approx(sum(quantityordered),0.5) over (partition by productline order by month_id rows between unbounded preceding and current row) as median_sale,
  variance(sum(quantityordered)) over (partition by productline order by month_id rows between unbounded preceding and current row) as Varience_sale,
  max(sum(quantityordered)) over (partition by productline order by month_id rows between unbounded preceding and current row) as max_sale
from productssales
group by month_id, productline
order by productline, month_id

month_id,productline,total_quantity_ordered,avg_sale,median_sale,Varience_sale,max_sale
1,Classic Cars,2740,2740.0,2740,null,2740
2,Classic Cars,2586,2663.0,2586,11858.0,2740
3,Classic Cars,2458,2594.6666666666665,2586,19937.333333333336,2740
4,Classic Cars,2310,2523.5,2458,33550.333333333314,2740
5,Classic Cars,3057,2630.2,2586,82087.19999999998,3057
6,Classic Cars,1080,2371.8333333333335,2458,466189.7666666666,3057
7,Classic Cars,2028,2322.714285714286,2458,405380.23809523805,3057
8,Classic Cars,2224,2310.375,2310,348686.8392857142,3057
9,Classic Cars,1938,2269.0,2310,320507.9999999999,3057
10,Classic Cars,3818,2423.9,2310,524836.1000000001,3818


Databricks visualization. Run in Databricks to view.

#### 13.Identify consicutive quarters of sales decline for each product

In [0]:
%sql
with SalesLag as (
  select 
    qtr_id, 
    productcode,
    sum(sales) as total_sales,
    lag(sum(sales),1) over (partition by productcode order by qtr_id ) as Previous_Quarter_Sale,
    lag(sum(sales),2) over (partition by productcode order by qtr_id ) as Two_Quarters_Ago_Sales
from productssales
group by qtr_id, productcode
)
select 
  qtr_id,
  productcode,
  total_sales,
  Previous_Quarter_Sale,
  Two_Quarters_Ago_Sales
  from SalesLag
  where total_sales < Previous_Quarter_Sale and Previous_Quarter_Sale < Two_Quarters_Ago_Sales
  order by qtr_id, productcode


qtr_id,productcode,total_sales,Previous_Quarter_Sale,Two_Quarters_Ago_Sales
3,S10_4757,15525.760000000002,28118.000000000004,30667.030000000002
3,S10_4962,21557.940000000002,26600.789999999997,29109.129999999997
3,S12_1108,34228.520000000004,36597.58,42518.53999999999
3,S12_1666,22350.920000000002,29229.57,29907.78
3,S12_3148,23269.35,25896.08,30764.62
3,S12_4675,16340.59,21892.25,25090.4
3,S18_1097,20474.579999999998,23156.0,28432.050000000003
3,S18_1749,14919.2,29819.699999999997,31003.16
3,S18_1889,14673.890000000001,16225.439999999999,17179.46
3,S18_2248,4799.64,10000.23,17959.859999999997


Databricks visualization. Run in Databricks to view.

#### 15. Identify top 10% products by sales in eac quarter


In [0]:
%sql
with SalesRank as (
    select 
    qtr_id,
    productcode,
    sales, 
    percent_rank() over(partition by qtr_id order by sales desc) as Sales_Percent_Rank
    from productssales
)
select 
qtr_id,
productcode,
sales 
from SalesRank
where Sales_Percent_Rank <= 0.1
order by qtr_id, Sales_Percent_Rank

qtr_id,productcode,sales
1,S700_1691,10066.6
1,S24_4278,10039.6
1,S10_4698,9774.03
1,S18_2795,9534.5
1,S18_1342,9240.44
1,S50_1514,8977.05
1,S18_3320,8935.5
1,S18_2248,8884.8
1,S12_1108,8690.36
1,S12_1108,8602.92


In [0]:
%sql
with SalesPercentile as (
    select 
    qtr_id,
    productcode,
    sales, 
    NTILE(100) over(partition by qtr_id order by sales desc) as Percentile
    from productssales
)
select 
qtr_id,
productcode,
sales,
case when Percentile <= 25 then 'Low'
when Percentile between 26 and 75 then 'Medium'
else 'High' end 
from SalesPercentile

qtr_id,productcode,sales,CASE WHEN (Percentile <= 25) THEN Low WHEN ((Percentile >= 26) AND (Percentile <= 75)) THEN Medium ELSE High END
1,S700_1691,10066.6,Low
1,S24_4278,10039.6,Low
1,S10_4698,9774.03,Low
1,S18_2795,9534.5,Low
1,S18_1342,9240.44,Low
1,S50_1514,8977.05,Low
1,S18_3320,8935.5,Low
1,S18_2248,8884.8,Low
1,S12_1108,8690.36,Low
1,S12_1108,8602.92,Low


#### Month with highest sales in each year


In [0]:
%sql
select year_id, month_id, total_sales, Rank 
from (select year_id, month_id, total_sales, dense_rank() over(partition by year_id order by total_sales desc)
as rank from( select year_id, month_id, sum(sales) as total_sales from productssales group by year_id, month_id
) as AggregatedSales 
) as RankedSales
where rank = 1 order by year_id, month_id

year_id,month_id,total_sales,Rank
2003,11,1029837.6600000001,1
2004,11,1089048.0100000005,1
2005,5,457861.05999999965,1
